In [41]:
import yfinance as yf
import pandas as pd
import warnings
import numpy as np
from typing import List
warnings.filterwarnings("ignore")

In [42]:
test_output = yf.download(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL','^GSPC'], start="2015-01-01", end="2020-02-21")["Close"].pct_change().dropna()
test_output.head()

[*********************100%%**********************]  6 of 6 completed


Ticker,AAPL,AMZN,GOOGL,META,NFLX,^GSPC
Date,,,,,,
2015-01-05,-0.028172,-0.020517,-0.019054,-0.016061,-0.050897,-0.018278
2015-01-06,0.000094,-0.022833,-0.024679,-0.013473,-0.017121,-0.008893
2015-01-07,0.014022,0.010600,-0.002941,0.000000,0.005192,0.011630
2015-01-08,0.038422,0.006836,0.003484,0.026658,0.022188,0.017888
2015-01-09,0.001073,-0.011749,-0.012211,-0.005628,-0.015458,-0.008404


In [44]:
test_output.pct_change()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,^GSPC
Date,,,,,,
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,-1.003342,0.112883,0.295239,-0.161126,-0.663624,-0.513436
2015-01-07,147.953941,-1.464222,-0.880833,-1.000000,-1.303252,-2.307683
2015-01-08,1.740105,-0.355077,-2.184693,inf,3.273661,0.538136
2015-01-09,-0.972086,-2.718633,-4.504803,-1.211122,-1.696665,-1.469794
...,...,...,...,...,...,...
2020-02-13,-1.299863,-2.096400,-1.607980,-0.344623,-0.783724,-1.252288
2020-02-14,-1.034588,0.487727,-2.022602,-0.567908,-1.716811,-2.130693
2020-02-18,-75.340957,-2.396411,-0.867509,2.463900,-8.399406,-2.583884


In [54]:

class TickerAnalytics:
    """A class to calculate and tabulate financial metrics for a list of tickers."""
    
    def __init__(self, tickers: List[str], start: str, end: str, benchmark_ticker: str = '^GSPC', rf_rate: float = 0.0,  interval: str = '1d'):
        """
        Initialize TickerAnalytics with a list of tickers, start and end dates, a benchmark ticker, risk-free rate, and data interval.
        
        Args:
            tickers (List[str]): List of ticker symbols.
            start (str): Start date in 'YYYY-MM-DD' format.
            end (str): End date in 'YYYY-MM-DD' format.
            benchmark_ticker (str): Ticker symbol of the benchmark. Defaults to '^GSPC'.
            rf_rate (float): Risk-free rate. Defaults to 0.0.
            interval (str): Data interval. Defaults to '1d'.
        """
        self.tickers = tickers
        self.benchmark_ticker = benchmark_ticker
        self.rf_rate = rf_rate
        self.data = self.get_data(start, end, interval)
        self.returns = self.calculate_returns()
        
        
    def get_data(self, start: str, end: str, interval: str) -> pd.DataFrame:
        """
        Download historical data for the tickers and benchmark (SP500).
        
        Args:
            start (str): Start date in 'YYYY-MM-DD' format.
            end (str): End date in 'YYYY-MM-DD' format.
            interval (str): Data interval.
            
        Returns:
            pd.DataFrame: DataFrame of historical data.
        """
        data = yf.download(self.tickers+[self.benchmark_ticker], start=start, end=end, interval=interval)["Close"]
        return data
    
    def calculate_returns(self) -> pd.DataFrame:
        """
        Calculate returns for the tickers and benchmark.
        pct_change() is used to calculate the percentage change between the current and a prior element.
        
        Returns:
            pd.DataFrame: DataFrame of returns.
        """
        return self.data.pct_change().dropna()
    
    def calculate_beta(self, ticker: str) -> float:
        """
        Calculate beta for a ticker.
        the covariance of the stock with the market divided by the variance of the market
        the 0,1 index is the covariance of the stock with the market
        the 1,1 index is the variance of the market
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Beta of the ticker.
        """
        cov = np.cov(self.returns[ticker], self.returns[self.benchmark_ticker])
        return cov[0][1]/cov[1][1]
    
    def calculate_alpha(self, ticker: str) -> float:
        """
        Calculate alpha for a ticker.
        the formula is the average return of the stock - the risk free rate - beta * (average return of the market - risk free rate)
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Alpha of the ticker.
        """
        beta = self.calculate_beta(ticker)
        market_return = self.returns[self.benchmark_ticker].mean() * 252
        individual_ticker_return = self.returns[ticker].mean() * 252
        return individual_ticker_return - (self.rf_rate + beta * (market_return - self.rf_rate))
    
    def calculate_sharpe_ratio(self, ticker: str) -> float:
        """
        Calculate Sharpe ratio for a ticker.
        the formula is the average return of the stock - the risk free rate / the standard deviation of the stock
        
        Args:
            ticker (str): Ticker symbol.
            
        Returns:
            float: Sharpe ratio of the ticker.
        """
        individual_ticker_return = self.returns[ticker].mean() * 252
        individual_tick_std = self.returns[ticker].std() * np.sqrt(252)
        return (individual_ticker_return - self.rf_rate)/individual_tick_std
    
    def tabulate_metrics(self) -> pd.DataFrame:
        """
        Tabulate the financial metrics for all tickers in a DataFrame.
        
        Returns:
            pd.DataFrame: DataFrame of financial metrics.
        """
        metrics = pd.DataFrame(index=self.tickers)
        for ticker in self.tickers:
            metrics.loc[ticker, "Return"] = self.returns[ticker].mean() * 252
            metrics.loc[ticker, "Standard Deviation"] = self.returns[ticker].std() * np.sqrt(252)
            metrics.loc[ticker, "Covariance with Market"] = np.cov(self.returns[ticker], self.returns[self.benchmark_ticker])[0][1]
            metrics.loc[ticker, "Beta"] = self.calculate_beta(ticker)
            metrics.loc[ticker, "Alpha"] = self.calculate_alpha(ticker)
            metrics.loc[ticker, "Sharpe Ratio"] = self.calculate_sharpe_ratio(ticker)
        return metrics

In [55]:
ticker_analytics = TickerAnalytics(['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL'], start="2023-01-30", end="2024-01-30")

[*********************100%%**********************]  6 of 6 completed


In [56]:
ticker_analytics.tabulate_metrics()

,Return,Standard Deviation,Covariance with Market,Beta,Alpha,Sharpe Ratio
META,1.085580,0.392639,0.000115,1.835569,0.693185,2.764826
AAPL,0.315602,0.199679,0.000070,1.122791,0.075579,1.580548
AMZN,0.526654,0.317442,0.000095,1.516632,0.202439,1.659056
NFLX,0.561961,0.375940,0.000078,1.251997,0.294318,1.494816
GOOGL,0.506840,0.293492,0.000088,1.397358,0.208123,1.726929
